In [1]:
%reload_ext autoreload
%autoreload 2
import colabexts
from colabexts.jcommon import *

jpath=os.path.dirname(colabexts.__file__)
jcom = f'{jpath}/jcommon.ipynb'
%run $jcom

%matplotlib inline
pd.set_option('display.max_colwidth', 40)
pd.set_option('display.max_rows', 16)


In [5]:
%%writefile ../suggestindex.py
#!/usr/bin/env python 

import sys, os, json, math, re, hashlib
from datetime import datetime, timedelta, time
import elasticsearch
from elasticsearch import Elasticsearch
import requests

#sys.path.append("/opt/utils")
#import filestoes, utils
#from utils import Map
from  mangorest.mango import webapi
import blogs.map

es         = Elasticsearch("http://localhost:9200")
INDEX_NAME = "suggest_index"

#--------------------------------------------------------------------------------
@webapi("/blogs/suggest")
def Suggest(q="", user="", prefs="", location="", **kwargs):
    r = es.search(index=INDEX_NAME, filter_path=[''], q=q, size=7)
    ret = [ f"{t['_source']['title']}" for t in r['hits']['hits'] ]
    ret = [k.replace("%20", " ") for k in list(set(ret))]

    if ( len(ret) < 10):
        response=Get3PSuggestions(q);
        for q in response.json()[1]:
            if ( len(ret) >= 10):
                break
            ret.append(q)

    ret = json.dumps(ret)
    
    print ("Suggest returning: ", ret[0:64])
    return ret
#-----------------------------------------------------------------------------------------------------------------
def InsertSuggestion( query="", user='unknonwn', es=es, **kwargs):
    query = query.strip() 
    if ( len(query) < 3 ):
        return 
    
    print(f'+++ indexing for {user}: {query}')
    id = hash(query)
    try:
        h = es.get(index=INDEX_NAME, id=id)
        print(f"Found an entry already for {query}")    
    except:
        h =None;
    
    if ( h is None):
        #userhash = hash(user)
        es.index(index=INDEX_NAME, id=id, 
                 body={
                    'title':query, 
                    'count': 0,
                    'id': id,      
                    'users': {},
                    'timeUTC': datetime.utcnow(), 
                    'timeLst': datetime.utcnow(), 
                })
        
        if ( len(query.split()) > 1 ): # Get suggestions from Google and insert it
            response=Get3PSuggestions(query);
            for q in response.json()[1]:
                print(f"==> Inserting Suggestions: {q}")
                id = hash(q)
                es.index(index= INDEX_NAME, id=id, body={
                    'title':q,  'count': 0,
                    'id': id,   
                    'users': {},
                    'timeUTC':  datetime.utcnow(), 
                    'timeLast': datetime.utcnow(), })
    else:
        u = h['_source']['users'].get(user,0) + 1
        body={
         "script": {
            "source" : f"ctx._source.count = ctx._source.count + 1; ctx._source.users['{user}'] = {u}",
             #"users" : {},
             "lang": "painless" },
          "query": { "terms": { "_id": [id] } },
            
        }
        h= es.update_by_query(index=INDEX_NAME,  body=body)
    return h;

#-------------------------------------------------------------------------------------------
# Search for some top 10 of last 10 searches for a users - 
# TODO: please review carefully
#
def seachSuggestedIndex(q="", IDS=[], users="".split('.'), es=es ):
    #lets make sure we got one entry in the system
    matchAll={'size': 30, "query": {"match_all": {}} }
    searchIDS = {"query": {"terms": { "_id": [IDS] }} }
                 
    # -> Search for all docs and print it
    if (len(IDS) > 0):
        ls = es.search(index=INDEX_NAME,body= searchIDS)
    elif( q is not None):
        ls = es.search(index=INDEX_NAME,q=q);
    else:
        print("Searching all ...")
        ls = es.search(index=INDEX_NAME,body= matchAll)
        
    for i,h in enumerate(ls['hits']['hits']):
        s = Map(h['_source'])
        ui= s.userid if s.userid else ""
        ta= ":".join ([str(k) for k in s.keys()])
        co= s.count if 'count' in s else "NA"
        print(f"{h['_id']:26} {i}/{ls['hits']['total']} count:{co:3} {ui:3} {s.utc} {s.title[0:10]} Tags: {ta}    ")
        
    return ls;

#-----------------------------------------------------------------------------------------------------------------
#Insert3PSuggestions(None, "Insight Rover", )
def Get3PSuggestions(query, proxies={'http':None, 'https': None}):
    #bq = urllib.parse.quote(query)
    #bs=f"http://api.bing.com/osjson.aspx?query={bq}"
    gq = re.subn('[ ]+','+',query)[0]
    gs=f"http://google.com/complete/search?q={gq}&output=chrome&hl=en"
    #print(f"{bs}\n{gs}")

    response=requests.get(gs,proxies=proxies, verify=False)    
    for c in response.json()[1]:
        print(c)
    return response

#-----------------------------------------------------------------------------------------------------------------
#es.indices.delete(index=INDEX_NAME)
def delFromSuggestedInex( es, title= 'stuff', id=None ):
    id = id if id else hash(title)
    timet = int(time.time())
    es.delete(index=INDEX_NAME, id=id)

#InsertSuggestion("First Some Test", user='sada')

Overwriting ../suggestindex.py


In [7]:
import sys
sys.path.append("/opt/utils")
import filestoes, utils
from utils import Map
from  mangorest.mango import webapi
from services.gen import suggestindex

#--TESTS---------------------------------------------------------------------------------------------------------------
#InsertSuggestion('python Loop')
h=suggestindex.seachSuggestedIndex('tunnel')
h


-4450120421605036574       0/{'value': 2, 'relation': 'eq'} count:  0     None tunnel Tags: title:count:id:users:timeUTC:timeLst    
3659739288941941430        1/{'value': 2, 'relation': 'eq'} count:  0     None tunnel Tags: title:count:id:users:timeUTC:timeLst    


{'took': 2,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 2, 'relation': 'eq'},
  'max_score': 0.4700036,
  'hits': [{'_index': 'suggest_index',
    '_id': '-4450120421605036574',
    '_score': 0.4700036,
    '_source': {'title': 'tunnel',
     'count': 0,
     'id': -4450120421605036574,
     'users': {},
     'timeUTC': '2022-03-12T18:00:41.424119',
     'timeLst': '2022-03-12T18:00:41.424122'}},
   {'_index': 'suggest_index',
    '_id': '3659739288941941430',
    '_score': 0.4700036,
    '_source': {'title': 'tunnel',
     'count': 0,
     'id': 3659739288941941430,
     'users': {},
     'timeUTC': '2022-03-12T18:21:17.214756',
     'timeLst': '2022-03-12T18:21:17.214759'}}]}}

# CHECK and Delete Below

In [1]:
#h['hits']['hits'][0]['_source']['users'].get(users,0) + 1

In [144]:
'''
def updateAllDocs(es, index='suggest_index', doc_type='text'):
    body={
        "script": {
            #"source": "ctx",
            "source": f"ctx._source.utc  = {int(time.time())}",
             "lang": "painless"
        },
        "query": {"match_all": {}}
    }
    try:
        ls=es.update_by_query(index=index,  body=body)
        return ls
    except Exception as e:
        print(e)
        return None                  
        

#--------------------------------------------------------------------------------
#Call this as "http://localhost:8001/api/modules.test.test/"
#
def getALL(es, index, docType, IDS=[], q=None, remove=['_source', 'id', '_index', '_type'],  need=[], maxlen=32):
    #lets make sure we got one entry in the system
    matchAll={"query": {"match_all": {}} }
    searchIDS = {"query": {"terms": { "_id": [IDS] }} }
                 
    # -> Search for all docs and print it
    if (len(IDS) > 0):
        ls = es.search(index=index, _source_exclude=['data',], doc_type = docType, body= searchIDS)
    elif( q is not None):
        ls = es.search(index=index, _source_exclude=['data',], doc_type = docType, q=q);
    else:
        ls = es.search(index=index, _source_exclude=['data',], doc_type = docType, body= matchAll, size=25)
        
    k1=[]
    k2=[];
    ths=""
    #need=['_source.utc', '_source.dtmt', '_type']
    
    if ( len(need) > 0):
        k1=[n for n in need if n.find ('.') < 0]
        k2=[n.split('.')[1] for n in need if n.find ('.') > 0]
    else:
        for h in ls['hits']['hits']:
            for k in h.keys():
                if ( k in remove):
                    continue;
                if ( k not in k1): 
                    k1.append(k)
            for k in h['_source'].keys():
                if ( k in remove):
                    continue;
                if ( k not in k2): 
                    k2.append(k)

    for k in k1 + k2:
        ths +=f"<th>{k}</th>"

    ret ="<table><tr>"+ths+"</tr>\n"
    for h in ls['hits']['hits']:
        tds="<tr>"
        for k in k1:
            tds += f"<td>{h[k]}</td>"
        for k in k2:
            v = h['_source'][k] if k in h['_source'] else ""
            if (type(v) == str):
                v = v[0:maxlen]
            tds += f"<td>{v}</td>"
        ret += tds + "</tr>\n";

    return ret


def searchSuggestIndex(request, proxies={'http':None, 'https': None}):
    parameters = dict(request.POST)
    i = request.GET.get( 'index', "suggest_index")
    t = request.GET.get( 'type', "text")
    q = request.GET.get( 'q', "")
    u = request.GET.get( 'user', "")
    p = request.GET.get( 'prefs', "")

    response= SearchGoogle(q, proxies)
    ret = FormatGoogle(q, response)
    return(HttpResponse(ret))

#ls = getALL(es, "suggest_index", 'text', IDS=[], q=None)
#HTML(ls)'''